# *Module 4 - LSTM Model*
___

# 1. Project summary
___

This is part of a project to build a sentiment classifier trained on Yelp review data (https://www.yelp.com/dataset). The project has been divided into several modules to perform different parts of the analysis, e.g., data cleaning, data processing, and model training. The goal is to predict the sentiment of a document; while using Yelp reviews of businesses, the 1-5 star rating acts as a proxy for sentiment, and the written Yelp review as the document text. The project is written in Python on Jupyter notebooks and makes use of a range of data science tools like pandas, spaCy, word2vec, and keras. My motivation in starting this project is to build my skillset, learn new tools, and improve as a data scientist. It is an ongoing project and may see many updates/iterations.

# 2. Module overview
___

## Goal
- The goal of this module is to train a classifier to identify the sentiment of a text document using Yelp review data
    - Data Splitting:
        - Split data into training and testing samples
    - Modeling:
        - Define the structure of the model using LSTM
    - Training:
        - Train the model using the encoded Yelp data
    - Evaluation
        - Evvaluate the performance of the model

## Dataset
- `yelp-dataset-reviews-encoded` (Kaggle: https://www.kaggle.com/datasets/gabrielmadigan/yelp-dataset-reviews-encoded)
- `yelp-dataset-reviews-nlp` (Kaggle: https://www.kaggle.com/datasets/gabrielmadigan/yelp-dataset-reviews-nlp)
- Derived from: `yelp-dataset`
    - Kaggle: https://www.kaggle.com/datasets/yelp-dataset/yelp-dataset
    - Yelp: https://www.yelp.com/dataset
    - Description from the kaggle dataset page:
    > This dataset is a subset of Yelp's businesses, reviews, and user data. It was originally put together for the Yelp Dataset Challenge which is a chance for students to conduct research or analysis on Yelp's data and share their discoveries. In the most recent dataset you'll find information about businesses across 8 metropolitan areas in the USA and Canada.
- This dataset has been prepared for this module following cleaning, reduction, natural language processing, and numerical encoding of the original Yelp dataset.

## Libraries

- **NumPy** 
    - A fundamental Python package for performing scientific analysis and computation, used to transform data
    - Webpage: https://numpy.org/
- **Pandas** 
    - Used to read, load, store, inspect, process, and save the data
    - Webpage: https://pandas.pydata.org/
- **Keras** 
    - A powerful library for training deep learning models used to train the classifier 
    - Webpage: https://keras.io/

## Output
- Model states are saved as `.keras` files after each training epoch 
- Naming convention for files is `model_checkpoint_epochXX_lossY.YY.keras` where `XX` is the number epochs the model was trained on and `Y.YY` is the validation loss

# 3. Import libraries
___

- I use `numpy` to pass data as `ndarray` objects to our model
- I will be importing `pandas` to read, load, and handle the data
- For plotting during evaluation I'll make use of the `matplotlib` and `seaborn` libraries
- The `wordcloud` library will allow us to construct a word cloud from the review text for each predicted class for validation
- I only import `sklearn` to split the data into training and testing sets
- The `keras` library will be used to build, train, and evaluate the classifier
- The `tqdm` package is useful for displaying progress bars while processing data (https://tqdm.github.io/)
- The module from `IPython` ensures every command in a cell is displayed, which saves me from having to write lots of print statements.

In [ ]:
# Libraries for reading, handling, manipulating and visualizing data
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from wordcloud import WordCloud

# keras modules for training and evaluating neural networks
import keras
from keras import Sequential, Input, metrics
from keras.layers import LSTM, Dense
from keras.callbacks import ModelCheckpoint

# Settings for displaying commands in a cell
from IPython.core.interactiveshell import InteractiveShell

- We can ignore the warning above
- Some settings for the notebook that aid with analysis

In [ ]:
# Display output of every command in a cell
InteractiveShell.ast_node_interactivity = 'all'

# set default seaborn theme for plots
#sns.set_theme()

# 4. Read data
___

- We will read in two files: the encoded data from the `yelp-dataset-reviews-encoded` dataset and the lemmatized data from `yelp-dataset-reviews-nlp` dataset
- The former contains the encoded data needed for training and evaluating the model
- The latter is only used to help sanity-check the performance of the model

In [ ]:
# Read and load the encoded Yelp review data as a pandas dataframe
df_encoded = pd.read_json('/kaggle/input/yelp-dataset-reviews-encoded/encoded_data.json', lines=True)

#  Read and load the lemmatized Yelp review data as a pandas dataframe
df_lemmatized = pd.read_json('/kaggle/input/yelp-dataset-reviews-nlp/lemmatized_data.json', lines=True)

# Combine the two dataframes
df = pd.concat([df_lemmatized, df_encoded], axis=1)

# Let's also add a label column to our df
df['label'] = np.where(df['stars'] <= 3, 'negative', 'positive')

# Inspect the size and data-types of the df
df.info()

# Inspect the first few rows
df.head()

- The dataset correctly contains 20k entries
- The data have been processed (cleaning, subsampling, tokenization, etc.)
- Text data have been lemmatized and encoded as sequences (uniform in length) of word embeddings
- Labels have been one-hot encoded corresponding to "positive" (4-5 stars) and "negative" (1-3 stars) sentiment

# 5. Data splitting
___

- Prior to training, we split the data into two sets: a training set and a testing set
- The training set will of course be passed to the model during training (where the weights will be updated)
- The testing set will be passed to the model while in a frozen state, without updating the weights, to asses the performance of the trained model on new data
- The split will be 8/2 training/testing
- We will also shuffle the data, but using a seed value to guarentee reproducibility

In [ ]:
TEST_SPLIT = 0.2
RANDOM_SEED = 0
df_train, df_test = train_test_split(df, test_size=TEST_SPLIT, shuffle=True, random_state=RANDOM_SEED)

df_train.head()
df_test.head()

- Keras requires us to pass the data as numpy arrays rather than pandas dataframes, so let's create numpy arrays for our input and label data
- N.B. The data will remain in their shuffled order, which ensures that we can merge the model outputs back into our original shuffled dataframes for evaluation

In [ ]:
# Convert encoded inputs and labels from pandas series to numpy arrays for training/evaluation
train_input = np.stack(df_train['word_embeddings'].values)
train_labels = np.stack(df_train['onehot_label'].values)
test_input = np.stack(df_test['word_embeddings'].values)
test_labels = np.stack(df_test['onehot_label'].values)

# Inspect the numpy arrays
print('Training set')
print('input data')
train_input.dtype
train_input.shape

print('label data')
train_labels.dtype
train_labels.shape

print('Testing set')
print('input data:')
test_input.dtype
test_input.shape

print('label data:')
test_labels.dtype
test_labels.shape

# 5. Model training
___

- Now we are ready to build our model
- We will use a type of Recurrent Neural Network (RNN) called Long Short Term Memory (LSTM)
- LSTM solves the exploding/vanishing gradient problem seen in vanilla RNNs by tracking long-term memory and short-term memory in parallel, where long-term memory does not get operated on by any weights that might cause it to explode/vanish when performing backpropagation
- Before we can build our model, we need to set some of the fixed parameters that define our model

In [ ]:
BATCH_SIZE = 32 # samples in each batch during training and evaluation
EMBEDDING_DIM = 100 # dimension of each word embedding
SEQUENCE_LENGTH = 150 # length of padded sequences of word embeddings
NUM_CLASSES = 2 # number of class labels for one-hot encoding
LSTM_LAYERS = 100 # number of recurrent layers in the LSTM RNN

- Now we can construct our model using a LSTM network
- We start with a `Sequential` object imported from the Keras library
- Then we add our input dimensions using the parameters we defined
- Next, add the LSTM network, specifying the number of recurrent layers
- Finally, we need to map the output of our LSTM RNN to the target dimension (labels), which we do using a fully-connected dense network
- The softmax activation function will map the logits of our output into probabilies for each class

In [ ]:
# Construct a LSTM model
model = Sequential()
model.add(keras.Input(shape=(SEQUENCE_LENGTH, EMBEDDING_DIM)))
model.add(LSTM(LSTM_LAYERS))
model.add(Dense(NUM_CLASSES, activation='softmax'))
model.summary()

- To train our model, we need to select a loss function to minimize after backpropogation and an optimization algorithm to perform when updating the model weights
- We will use cross-entropy for the loss function as it works well for classification tasks
- The optimizer we will use is Adam which is widely used and performs well in most situations
- We will also add an accuracy metric to track the accuracy of the model during training
- Model accuracy will be determined by assigning the prediction to the class with the highest probability, e.g., if the output is (0.75, 0.25), the first element has the higher probability (0.75) which corresponds to the "positive" class (1.0, 0.0)

In [ ]:
# Compile the model and define the loss function and optimizer
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=[metrics.CategoricalAccuracy()])

- One more step: we define the training parameters
- Epochs determines how many passes over the entire dataset the model will take in training
- Validation split: the fraction of training data that will be split off to form a validation set
- We want to save the model at each epoch, so define a "checkpoint" to save the model at after each epoch using a keras callback object

In [ ]:
# Train the model on the training set
EPOCHS = 20 # Train the model over some number of epochs (passes over entire dataset)
VALIDATION_SPLIT = 0.2 # Split off a validation set from the training data

# Checkpoint object
checkpoint_filepath = '/kaggle/working/model_checkpoint_epoch{epoch:02d}.keras'
model_checkpoint = keras.callbacks.ModelCheckpoint(checkpoint_filepath, 
                                monitor='val_accuracy', 
                                verbose=0, 
                                save_best_only=False, 
                                save_weights_only=False, 
                                mode='auto', 
                                save_freq='epoch',
                                initial_value_threshold=None,
                               )

- Finally, we can train the model and track the accuracy

In [ ]:
history = model.fit(train_input, 
                    train_labels, 
                    batch_size=BATCH_SIZE, 
                    epochs=EPOCHS, 
                    validation_split=VALIDATION_SPLIT, 
                    validation_batch_size=BATCH_SIZE, 
                    callbacks=[model_checkpoint]
                   )

- It will be easier to interpret the model's performance during training with a few plots

In [ ]:
# Plot the model accuracy per training epoch in training and validation data

# Create a subplot
fig, ax = plt.subplots(figsize=(10,5))

# Plot the data as a scatter plot, specify data labels for legend
ax.scatter(range(1, EPOCHS+1), history.history['categorical_accuracy'], label='Training', zorder=2)
ax.scatter(range(1, EPOCHS+1), history.history['val_categorical_accuracy'], color='#DC0000', label='Validation', zorder=2)

# Format the plots here

# Set plot title text and location
ax.set_title('LSTM models', loc='right')

# Set x- and y-axis label text and locations
ax.set_xlabel('Training epochs', labelpad=10, fontsize=14, loc='right')
ax.set_ylabel('Accuracy', labelpad=10, fontsize=14, loc='top')

# Add minor ticks
ax.minorticks_on()

# Set the length of x- and y-axis major and minor ticks, position the ticks inside the plot, and add ticks to the top and right side of the plot
ax.tick_params(axis="x", which='major', direction='in', length=8, right=True, top=True)
ax.tick_params(axis="x", which='minor', direction='in', length=4, right=True, top=True)
ax.tick_params(axis="y", which='major', direction='in', length=8, right=True, top=True)
ax.tick_params(axis="y", which='minor', direction='in', length=4, right=True, top=True)

# Set which x- and y-axis values to display (and major ticks, correspondingly)
ax.locator_params(axis='y', nbins=6)
ax.set_xticks(np.linspace(0, 20, 11).astype(int))

# Add grid lines
ax.grid(which='major', axis='x', color='black', linestyle='dotted', alpha=1.0, zorder=0)
ax.grid(which='major', axis='y', color='black', linestyle='dotted', alpha=1.0, zorder=0)

# Add a legend
ax.legend(fancybox=False, framealpha=1.0, edgecolor='black');


In [ ]:
# Plot the model's loss per training epoch in training and validation data

# Create a subplot
fig, ax = plt.subplots(figsize=(10,5))

# Plot the data as a line, specify data labels for legend
ax.scatter(range(1, EPOCHS+1), history.history['loss'], label='Training', zorder=2)
ax.scatter(range(1, EPOCHS+1), history.history['val_loss'], color='#DC0000', label='Validation', zorder=2)

# Format the plots here

# Set plot title text and location
ax.set_title('LSTM models', loc='right')

# Set x- and y-axis label text and locations
ax.set_xlabel('Training epochs', labelpad=10, fontsize=14, loc='right')
ax.set_ylabel('Loss', labelpad=10, fontsize=14, loc='top')

# Add minor ticks
ax.minorticks_on()

# Set the length of x- and y-axis major and minor ticks, position the ticks inside the plot, and add ticks to the top and right side of the plot
ax.tick_params(axis="x", which='major', direction='in', length=8, right=True, top=True)
ax.tick_params(axis="x", which='minor', direction='in', length=4, right=True, top=True)
ax.tick_params(axis="y", which='major', direction='in', length=8, right=True, top=True)
ax.tick_params(axis="y", which='minor', direction='in', length=4, right=True, top=True)

# Set which x- and y-axis values to display (and major ticks, correspondingly)
ax.locator_params(axis='y', nbins=6)
ax.set_xticks(np.linspace(0, 20, 11).astype(int))

# Add grid lines
ax.grid(which='major', axis='x', color='black', linestyle='dotted', alpha=1.0, zorder=0)
ax.grid(which='major', axis='y', color='black', linestyle='dotted', alpha=1.0, zorder=0)

# Add a legend
ax.legend(fancybox=False, framealpha=1.0, edgecolor='black');

- We can see the model accuracy improve in training data with each epoch before plateauing at around epoch 11
- The model accuracy with the validation set peaks at 3 epochs and then gradually declines
- This implies that there is strong bias toward training data in the model after 3 or so training epochs
- Likewise, looking at the loss after each epoch, we see that in training it continues to decrease (output more closely matches labels), while in the validation set the loss dips at 3 epochs and then continues to increase with more training epochs, further indicating that the model suffers from training bias after 3 epochs
- Therefore the best performing model is the one trained on 3 epochs
- Let's see how this model performs with the testing dataset we set aside

# 6. Model evaluation
___

- Time to evaluate our model
- Load the model state (weights and biases) saved after 3 training epochs back into the model

In [ ]:
# Load the saved model weights and biases
best_model_filepath = '/kaggle/working/model_checkpoint_epoch03.keras'
model.load_weights(best_model_filepath)

# Evaluate the model with the testing dataset
model.evaluate(test_input, test_labels, batch_size=BATCH_SIZE)

- Evaluating the model (trained with 4 epochs) with the test dataset returned an accuracy of 87.5 % which closely matches the accuracy of the validation set during training (88%)
- While not perfect, a testing accuracy of 87% shows that this is a particularly well-performing model!
- Now let's get the model's predictions with the training and testing sets for further evaluation

In [ ]:
# Get the predictions with our model for training and testing sets
train_pred = model.predict(train_input, batch_size=BATCH_SIZE)
test_pred = model.predict(test_input, batch_size=BATCH_SIZE)

- Now add the predictions back into the training and testing (split and shuffled) dataframes

In [ ]:
# Load the predictions into new dataframes (one for training one for testing)
df_train_pred = pd.DataFrame(train_pred, columns=['pos_pred', 'neg_pred'])
df_test_pred = pd.DataFrame(test_pred, columns=['pos_pred', 'neg_pred'])

# Add predictions as columns to training/testing dataframes
# The ordering of the predictions still matches the original shuffled train/test datasets if we convert to a list
df_train['pos_pred'] = list(df_train_pred['pos_pred'])
df_train['neg_pred'] = list(df_train_pred['neg_pred'])
df_test['pos_pred'] = list(df_test_pred['pos_pred'])
df_test['neg_pred'] = list(df_test_pred['neg_pred'])

# Now add the predicted sentiment label, based on a prediction threshold of 0.5
df_train['pred_label'] = np.where(df_train['pos_pred'] > 0.5, 'positive', 'negative')
df_test['pred_label'] = np.where(df_test['pos_pred'] > 0.5, 'positive', 'negative')

# Verify the size and data-types of the df
df_train.info()
df_test.info()

# Verify the first few rows
df_train.head()
df_test.head()


- It is helpful to look at the distribution of prediction values (probabilities) for each class to see how "confident" the model is in predicting each class

In [ ]:
# Plot the distribution of predictions (probabilities) for positive and negative sentiment data

# Create subplots with 2 rows
fig, ax = plt.subplots(nrows=2, sharex=True, figsize=(10,6))

# Plot the data as step histograms with 50 bins, a log-scale y-axis, and specify data labels for legend
# Training data
ax[0].hist(df_train.loc[df_train['label'] == 'positive']['pos_pred'], bins=50, log=True, histtype=u'step', label='Positive reviews') # positive data
ax[0].hist(df_train.loc[df_train['label'] == 'negative']['neg_pred'], bins=50, log=True, histtype=u'step', color='#DC0000', label='Negative reviews') # negative data

# Testing data
ax[1].hist(df_test.loc[df_test['label'] == 'positive']['pos_pred'], bins=50, log=True,  histtype=u'step', label='Positive reviews') # positive data
ax[1].hist(df_test.loc[df_test['label'] == 'negative']['neg_pred'], bins=50, log=True, histtype=u'step', color='#DC0000', label='Negative reviews') # negative data

# Format the plots here

# Set plot title text and location
ax[0].set_title('LSTM model (3 epochs)', loc='right')

# Set x- and y-axis label text and locations
ax[1].set_xlabel('Sentiment prediction (probability)', labelpad=10, fontsize=14, loc='right')
ax[0].set_ylabel('Frequency', labelpad=10, fontsize=14, loc='top')
ax[1].set_ylabel('Frequency', labelpad=10, fontsize=14, loc='top')

# Add minor ticks
ax[0].minorticks_on()
ax[1].minorticks_on()

# Set the length of x- and y-axis major and minor ticks, position the ticks inside the plot, and add ticks to the top and right side of the plot
ax[0].tick_params(axis="x", which='major', direction='in', length=8, right=True, top=True)
ax[0].tick_params(axis="x", which='minor', direction='in', length=4, right=True, top=True)
ax[0].tick_params(axis="y", which='major', direction='in', length=8, right=True, top=True)
ax[0].tick_params(axis="y", which='minor', direction='in', length=4, right=True, top=True)
ax[1].tick_params(axis="x", which='major', direction='in', length=8, right=True, top=True)
ax[1].tick_params(axis="x", which='minor', direction='in', length=4, right=True, top=True)
ax[1].tick_params(axis="y", which='major', direction='in', length=8, right=True, top=True)
ax[1].tick_params(axis="y", which='minor', direction='in', length=4, right=True, top=True)

# Set which x- and y-axis values to display (and major ticks, correspondingly)
#ax.locator_params(axis='y', nbins=6)
ax[0].set_xticks(np.linspace(0, 1, 11))
ax[1].set_xticks(np.linspace(0, 1, 11))

# Set x-axis range to display
ax[0].set_xlim(0, 1)
ax[1].set_xlim(0, 1)

# Add grid lines
ax[0].grid(which='major', axis='x', color='black', linestyle='dotted', alpha=1.0, zorder=0)
ax[0].grid(which='major', axis='y', color='black', linestyle='dotted', alpha=1.0, zorder=0)
ax[1].grid(which='major', axis='x', color='black', linestyle='dotted', alpha=1.0, zorder=0)
ax[1].grid(which='major', axis='y', color='black', linestyle='dotted', alpha=1.0, zorder=0)

# Add a legend
ax[0].legend(title='Training data', fancybox=False, framealpha=1.0, edgecolor='black')
ax[1].legend(title='Testing data', fancybox=False, framealpha=1.0, edgecolor='black');

- The distributions are strongly peaked toward a prediction probability of 1.0, indicating that the model performs well for both classes and on both datasets 
- There are more negative reviews predicted with low probability than there are positive (in the left tails), and vis versa for predictions with high probability (right peaks), indicating that the model performs better on positive reviews than negative
- This is expected as the data were biased toward posititve reviews

## Class accuracy, error, and confusion matrix

- We can turn our model into a binary classifier by assigning a review to the class that recieves a prediction greater than 0.5 (this was done automatically by Keras when computing the training, validation, and evaluation accuracies)
- Using this threshold we can compute the accuracy and error for each class, and a confusion matrix

In [ ]:
# Confusion matrix, accuracy, and erro for testing data

T = len(df_test) # total count (P + N)
P = len(df_test.loc[df_test['label'] == 'positive']) # Ground truth positive count
N = len(df_test.loc[df_test['label'] == 'negative']) # Ground truth negative count
PP = len(df_test.loc[df_test['pred_label'] == 'positive']) # Predicted positive count
PN = len(df_test.loc[df_test['pred_label'] == 'negative']) # Predicted negative count
TP = len(df_test.loc[(df_test['pred_label'] == 'positive') & (df_test['label'] == 'positive')]) # True positive count
FP = len(df_test.loc[(df_test['pred_label'] == 'positive') & (df_test['label'] == 'negative')]) # False positive count
TN = len(df_test.loc[(df_test['pred_label'] == 'negative') & (df_test['label'] == 'negative')]) # True negative count
FN = len(df_test.loc[(df_test['pred_label'] == 'negative') & (df_test['label'] == 'positive')]) # False negative count

T_rate = T/T # total rate (1.0)
P_rate = P/T # Grount truth positive rate
N_rate = N/T # Ground truth negative rate
PP_rate = PP/T # Predicted positive rate
PN_rate = PN/T # Predicted negative rate
TP_rate = TP/P # True positive rate
FP_rate = FP/N # False positive rate
TN_rate = TN/N # True negative rate
FN_rate = FN/P # False negative rate

# Confusion matrix of counts
confusion_matrix_counts = np.array([[T, PP, PN],
                                    [P, TP, FN],
                                    [N, FP ,TN]])

# Confusion matrix of rates
confusion_matrix_rates = np.array([[T_rate, PP_rate, PN_rate],
                                   [P_rate, TP_rate, FN_rate],
                                   [N_rate, FP_rate, TN_rate]])

# Let's also look at the accuracy and error for each class
T_acc = (TP+TN)/T # Total accuracy
T_err = (FP+FN)/T # Total error rate

# Print accuracy and errors
print(f'Model accuracy: {100*T_acc:.2f}%')
print(f'Model error: {100*T_err:.2f}%')

# Plot the confusion matrix of counts/rates

# Get the labels for each square
cm_names = ['Total', 'Positive', 'Negative', 'Positive', 'True Positive', 'False Negative', 'Negative', 'False Positive', 'True Negative']
cm_counts = [f'{count}' for count in confusion_matrix_counts.flatten()]
cm_rates = [f'{100*rate:.0f}%' for rate in confusion_matrix_rates.flatten()]
cm_labels = [ f'{name}\n{count}\n{rate}' for name, count, rate in zip(cm_names, cm_counts, cm_rates)]
cm_labels = np.asarray(cm_labels).reshape(3,3)

# Create a subplot
f, ax = plt.subplots(figsize=(6,6))

# Plot as a heatmap with annotations/values inside each square, with no sidebar
sns.heatmap(confusion_matrix_counts, annot=cm_labels, cmap='Blues', fmt='', cbar=False)

# Do not display x- and y-axis ticks/values
ax.set_xticks([])
ax.set_yticks([])

# Set the x- and y-axis labels
ax.set_xlabel('Predicted sentiment', labelpad=10, fontsize=14, loc='center')
ax.set_ylabel('True sentiment', labelpad=10, fontsize=14, loc='center');

- The model accuracy on test documents with positive sentiment is 94% while the error rate is 6%
- The model accuracy on test documents with negative sentiment is 73% while the error rate is 27%
- The model accuracy on test documents of any sentimnet is 87.5% while the error rate is 12.5% (consistent with the Keras evaluate method)

# Word Cloud

- As a final sanity-check, let's make a word cloud for all the text in each class prediction and see if this matches what we might expect

In [ ]:
# Join all the text into one string for each subset
text_pred_pos = ' '.join([' '.join(word) for word in list(df_test.loc[df_test['pred_label'] == 'positive']['lemmatized_text'])])
text_pred_neg = ' '.join([' '.join(word) for word in list(df_test.loc[df_test['pred_label'] == 'negative']['lemmatized_text'])])
text_label_pos = ' '.join([' '.join(word) for word in list(df_test.loc[df_test['label'] == 'positive']['lemmatized_text'])])
text_label_neg = ' '.join([' '.join(word) for word in list(df_test.loc[df_test['label'] == 'negative']['lemmatized_text'])])

# Create the word clouds for each subset
wc_pred_pos = WordCloud(background_color='white', colormap = 'RdBu').generate(text_pred_pos)
wc_pred_neg = WordCloud(background_color='white', colormap = 'RdBu').generate(text_pred_neg)
wc_label_pos = WordCloud(background_color='white', colormap = 'RdBu').generate(text_label_pos)
wc_label_neg = WordCloud(background_color='white', colormap = 'RdBu').generate(text_label_neg)

# Plotting

# Creat 4 subplots
fig, ax = plt.subplots(nrows=2, ncols=2, sharey=True, sharex=True, figsize=(10, 6))

# Plot the word clouds
ax[0,0].imshow(wc_label_pos)
ax[0,1].imshow(wc_pred_pos)
ax[1,0].imshow(wc_label_neg)
ax[1,1].imshow(wc_pred_neg)

# Set subplot titles text and location
ax[0,0].set_title('Positive sentiment (labeled)', loc='center')
ax[0,1].set_title('Positive sentiment (predicted)', loc='center')
ax[1,0].set_title('Negative sentiment (labeled)', loc='center')
ax[1,1].set_title('Negative sentiment (predicted)', loc='center')

# Turn off the axes
ax[0,0].axis("off")
ax[0,1].axis("off")
ax[1,0].axis("off")
ax[1,1].axis("off")

- The word clouds show that the most frequent words do not vary much between predicted and actual labels
- It is very clear that the positive sentiment includes words we would expect, like "good," "love," "amazing," and "delicious"
- The negative sentiment is harder to interpret---we still see expected words like "bad," and "wait,"---but we see more neutral than negative, like "food," "place," "service," and even positive words like "good"
- However, this makes some sense, as we included 3 star reviews in the negative class which could be seen as more neutral than negative, and even 1 and 2 star reviews may not carry as strong a sentiment as the positive reviews (despite being associated with such a low star rating)
- Also, there is far fewer data in the negative class than the positive which can impact these results

# 7. Discussion
___

Overall this project was a great success! The best model is able to achieve an overall accuracy of 87% which is decent performance given the training sample size and the limitations introduced by training on Yelp business reviews. In order to improve the model, several steps could be taken. 
1. Take a larger sample from the original dataset. I only used a small fraction to expidite the analysis, but given more time and memory resources I believe more training samples would improve the model accuracy. Also, selecting a subset with a uniform distribution of positive and negative sentiment would remove the heavy bias toward positive sentiment, if that was of interest. 
2. More fine tuning of the NLP. For example, using a larger corpus for the word2vec training or increasing the size of the word embeddings. 
3. Fine tune the model and training parameters. While I optimized on the number of training epochs, other parameters could be batch size and step size when finding the loss function minima. Also, I could experiment with other optimization algorithms and loss functions, although the Adam optimizer and cross-entropy are usually well suited for this type of task. For the model, the number of recurrent layers could potentially be optimized, along with the fully-connected layers. 
4. Changing the class definitions. I could experiment with how many stars are associated with each class, or even introduce a neutral class. This comes with additional challenges, and I think Yelp reviews are not the best source of text for that level of nuance. 
5. Removing high-frequency neutral words. As I noticed with the word clouds, both classes were still mostly neutral in tone. The words "place," "food," and "time" all appeared with very high frequency in all reviews. Removing these words from the text (like stop words) could potentially improve the performance of the model.
6. Finally, and what I think would offer the greatest improvement, is improve the data source and quality. Yelp reviews are not as well written as published text like Wikipedia, journals, books, etc., and include lots of typos, misspellings, odd/repeated punctuation, etc., which can confuse the spaCy NLP model. Better data cleaning can help but there will always be limits. The limited context of a Yelp review means there is not much variation in the content of the texts (this is most obvious when looking at the word clouds). For example, sentiment will almost exclusively involve either food taste, wait times, or service quality, which will bias the model. A corpus with a greater scope could result in a better model, e.g., IMDB movie reviews or Twitter/X tweets.